
# requirements.txt         # Lists required libraries

In [ ]:
import csv
import json
import yfinance as yf
import json
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# dump Listed ASX Data into JSON File

In [85]:
import csv
import json

# Download listed ASX company file from "https://www.asx.com.au/markets/trade-our-cash-market/directory"

def renew_industry_data():
    # This code will turn the company directory CSV to JSON for the fetch_data.py part
    with open('/Users/wayne/Daily Email Redo/ASX_Listed_Companies_07-02-2025_10-13-28_AEDT.csv','r') as f:
        csv_reader = csv.reader(f)  # CVS reader object
    
        next(csv_reader) # Skip the header
        data = {}
        for row in csv_reader:
            company_code = row[0]
            company_type = row[2]
    
            data[company_code] = {
                "industry": company_type,
                "market_cap": 0,
                "volume": 0,
                "average_daily_volume": 0,
                "number_of_shares": 0,
                "current_price": 0,
                "previous_close_price": 0,
                "price_change": "0.00%"
            }
            
    # Write data into JSON file
        with open('updated_industry_data.json', 'w') as json_file:
            json.dump(data, json_file,indent=4)

In [17]:
# JSON file check
with open('/Users/wayne/Daily Email Redo/updated_industry_data.json', 'r') as f:
    data = json.load(f)
    item = list(data.items())[0:5]
    print(item)

[('14D', {'industry': 'Capital Goods', 'volume': 0, 'last_price': 0, 'market_cap': 0, 'average_daily_volume': 0, 'price_change': '0.00%', 'previous_close_price': 0}), ('1AD', {'industry': 'Pharmaceuticals, Biotechnology & Life Sciences', 'volume': 0, 'last_price': 0, 'market_cap': 0, 'average_daily_volume': 0, 'price_change': '0.00%', 'previous_close_price': 0}), ('1AE', {'industry': 'Materials', 'volume': 0, 'last_price': 0, 'market_cap': 0, 'average_daily_volume': 0, 'price_change': '0.00%', 'previous_close_price': 0}), ('1AI', {'industry': 'Pharmaceuticals, Biotechnology & Life Sciences', 'volume': 0, 'last_price': 0, 'market_cap': 0, 'average_daily_volume': 0, 'price_change': '0.00%', 'previous_close_price': 0}), ('1CG', {'industry': 'Capital Goods', 'volume': 0, 'last_price': 0, 'market_cap': 0, 'average_daily_volume': 0, 'price_change': '0.00%', 'previous_close_price': 0})]


# fetch_data.py            # Handles API requests and data retrieval    

In [79]:
# TESTING YFINANCE'S FUNCTION
# !pip install yfinance
import yfinance as yf

ticker = yf.Ticker('I88.AX')
print(ticker.history(period='1y'))
print(ticker.financials)
print(ticker.actions)
info = ticker.get_info()
print(json.dumps(info, indent=4))

                            Open   High    Low  Close  Volume  Dividends  \
Date                                                                       
2024-02-07 00:00:00+11:00  0.270  0.280  0.260  0.265  230105        0.0   
2024-02-08 00:00:00+11:00  0.265  0.280  0.260  0.265  227343        0.0   
2024-02-09 00:00:00+11:00  0.280  0.285  0.255  0.285  242762        0.0   
2024-02-12 00:00:00+11:00  0.280  0.280  0.265  0.265   33543        0.0   
2024-02-13 00:00:00+11:00  0.265  0.265  0.255  0.265  111733        0.0   
...                          ...    ...    ...    ...     ...        ...   
2025-02-03 00:00:00+11:00  0.695  0.695  0.650  0.655  194374        0.0   
2025-02-04 00:00:00+11:00  0.655  0.655  0.655  0.655       0        0.0   
2025-02-05 00:00:00+11:00  0.655  0.655  0.655  0.655       0        0.0   
2025-02-06 00:00:00+11:00  0.650  0.655  0.615  0.620  319919        0.0   
2025-02-07 00:00:00+11:00  0.600  0.620  0.580  0.610  128005        0.0   

           

In [119]:
import yfinance as yf
import json
import time

def fetch_data(asx_code, max_retries=3):
    """Fetches stock data from Yahoo Finance with retry logic."""
    retries = 0
    while retries < max_retries:
        try:
            ticker = yf.Ticker(asx_code + ".AX")
            info = ticker.info  # Store the ticker info once to avoid multiple API calls

            market_cap = info.get("marketCap", 0)
            volume = info.get("volume", 0)
            average_daily_volume = info.get("averageVolume", 0)
            number_of_shares = info.get("sharesOutstanding", 0)
            current_price = info.get("currentPrice", 0)
            previous_close_price = info.get("previousClose", 0)

            price_change = "0.00%"
            if previous_close_price:
                price_change = "{:.2f}%".format(100 * (current_price - previous_close_price) / previous_close_price)

            return {
                "market_cap": market_cap,
                "volume": volume,
                "average_daily_volume": average_daily_volume,
                "number_of_shares": number_of_shares,
                "current_price": current_price,
                "previous_close_price": previous_close_price,
                "price_change": price_change
            }

        except Exception as e:
            retries += 1
            print(f"Error fetching {asx_code} (Attempt {retries}/{max_retries}): {e}")

            if "Too many requests" in str(e):
                wait_time = 10 * retries  # Wait 10s, then 20s, then 30s, etc.
                print(f"Rate limit hit! Waiting {wait_time} seconds before retrying...")
                time.sleep(wait_time)
            else:
                time.sleep(3)  # General wait for other errors

    print(f"Failed to fetch data for {asx_code} after {max_retries} retries.")
    return None

# Load the last processed index
def load_previous_state():
    try:
        with open('last_processed_index.json', 'r') as f:
            return json.load(f).get('last_index', 0)
    except FileNotFoundError:
        return 0

def save_last_processed_index(index):
    with open('last_processed_index.json', 'w') as f:
        json.dump({"last_index": index}, f)

def update_stock_data():
    try:
        # Load the last processed index
        last_processed_index = load_previous_state()

        with open('updated_industry_data.json', 'r') as json_file:
            company_data = json.load(json_file)

        for index, (company_code, data) in enumerate(company_data.items()):
            # Skip processed companies based on the saved index
            if index < last_processed_index:
                continue

            print(f"Fetching data for {company_code} (Index: {index + 1})")
            updated_tickers = fetch_data(company_code)
            if updated_tickers:
                company_data[company_code].update(updated_tickers)
                with open("Today's tickers report.json", 'w') as json_file:
                    json.dump(company_data, json_file, indent=4)

                # Save the index after processing each company
                save_last_processed_index(index + 1)

    except Exception as e:
        print(f"Error in update_stock_data: {e}")


Fetching data for 88E (Index: 18)
Fetching data for 8CO (Index: 19)
Fetching data for 8IH (Index: 20)
Fetching data for 8VI (Index: 21)
Fetching data for A11 (Index: 22)
Fetching data for A1G (Index: 23)
Fetching data for A1M (Index: 24)
Fetching data for A1N (Index: 25)
Fetching data for A2M (Index: 26)
Fetching data for A3D (Index: 27)
Fetching data for A4N (Index: 28)
Fetching data for A8G (Index: 29)
Fetching data for AAC (Index: 30)
Fetching data for AAI (Index: 31)
Fetching data for AAJ (Index: 32)
Fetching data for AAL (Index: 33)
Fetching data for AAM (Index: 34)
Fetching data for AAP (Index: 35)
Fetching data for AAR (Index: 36)
Fetching data for AAU (Index: 37)
Fetching data for ABA (Index: 38)
Fetching data for ABB (Index: 39)
Fetching data for ABE (Index: 40)
Fetching data for ABG (Index: 41)
Fetching data for ABV (Index: 42)
Fetching data for ABX (Index: 43)
Fetching data for ABY (Index: 44)
Fetching data for ACE (Index: 45)
Fetching data for ACF (Index: 46)
Fetching data 

In [105]:
with open('updated_industry_data.json', 'r') as json_file:
    company_list = json.load(json_file)
    print(company_list)

{'14D': {'industry': 'Capital Goods', 'market_cap': 0, 'volume': 0, 'average_daily_volume': 0, 'number_of_shares': 0, 'current_price': 0, 'previous_close_price': 0, 'price_change': '0.00%'}, '1AD': {'industry': 'Pharmaceuticals, Biotechnology & Life Sciences', 'market_cap': 0, 'volume': 0, 'average_daily_volume': 0, 'number_of_shares': 0, 'current_price': 0, 'previous_close_price': 0, 'price_change': '0.00%'}, '1AE': {'industry': 'Materials', 'market_cap': 0, 'volume': 0, 'average_daily_volume': 0, 'number_of_shares': 0, 'current_price': 0, 'previous_close_price': 0, 'price_change': '0.00%'}, '1AI': {'industry': 'Pharmaceuticals, Biotechnology & Life Sciences', 'market_cap': 0, 'volume': 0, 'average_daily_volume': 0, 'number_of_shares': 0, 'current_price': 0, 'previous_close_price': 0, 'price_change': '0.00%'}, '1CG': {'industry': 'Capital Goods', 'market_cap': 0, 'volume': 0, 'average_daily_volume': 0, 'number_of_shares': 0, 'current_price': 0, 'previous_close_price': 0, 'price_change

# process_data.py          # Sorts, filters, and formats stock data  

In [214]:
import csv

def write_to_csv_file(filename, sorted_stocks):
    with open(filename, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([
                         'ASX_Code', 
                         'industry', 
                         'market_cap',
                         'volume',
                         'average_daily_volume',
                         'number_of_shares',
                         'current_price',
                         'Commoprevious_close_pricedities',
                         'price_change'
        ])
        
        for stock in sorted_stocks:
            writer.writerow([
                            stock[0],
                            stock[1].get('industry', 'N/A'),
                            stock[1].get('market_cap', 'N/A'),
                            stock[1].get('volume', 'N/A'),
                            stock[1].get('average_daily_volume', 'N/A'),
                            stock[1].get('number_of_shares', 'N/A'),
                            stock[1].get('current_price', 'N/A'),
                            stock[1].get('previous_close_price', 'N/A'),
                            stock[1].get('price_change', 'N/A')
            ])

def process_data():
    
    with open("Today's tickers report.json", "r") as f:
        data_collection = json.load(f)

    # Separate stocks into two lists: with numeric volume and with 0 volume
    numeric_volume_stocks = [(symbol, data) for symbol, data in data_collection.items() if data.get("volume") != 0]
    non_volumne_stocks = [(symbol, data) for symbol, data in data_collection.items() if data.get("volume") == 0]
    
    sorted_numeric_stocks = sorted(numeric_volume_stocks, key=lambda x:x[1].get("volume", 0),reverse=True)
    sorted_volume_stocks = sorted_numeric_stocks + non_volumne_stocks
    
    # Sort dictionary with price change
    sorted_stocks_positive = sorted(data_collection.items(), key=lambda x:float(x[1].get("price_change",0).strip("%")), reverse=False)
    sorted_stocks_negative = sorted(data_collection.items(), key=lambda x:float(x[1].get("price_change", 0).strip("%")), reverse=True)

    write_to_csv_file('sorted_volume_stocks.csv', sorted_volume_stocks)
    write_to_csv_file('sorted_stocks_positive.csv', sorted_stocks_positive)
    write_to_csv_file('sorted_stocks_negative.csv', sorted_stocks_negative)

process_data()


# email_sender.py          # Sends emails with stock reports  


In [238]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def generate_email_body():
    with open("Today's tickers report.json", 'r') as f:
        data_collection = json.load(f)

        sorted_stocks_positive = sorted(data_collection.items(), key=lambda x:float(x[1].get('price_change').strip('%')), reverse=True)
        sorted_stocks_negative = sorted(data_collection.items(), key=lambda x:float(x[1].get('price_change').strip('%')), reverse=False)

        top_25_positive = sorted_stocks_positive[:25]
        top_25_negative = sorted_stocks_negative[:25]

        email_body = (
                "Hello,\n\nHere is today's list of top-volume stocks with the price change. \n\n"
                "The stocks with the most significant price changes are listed below.\n\n" 
                "For the detailed information, please refer to the attached spreadsheets:\n\n"        
        )

        email_body += "Top 25 stocks with positive price change:\n\n"
        for stock in top_25_positive:
            email_body += f"Stock:{stock[0]}, Price change: {stock[1]['price_change']}\n"
            
        email_body += "\nTop 25 stocks with negative price change:\n\n"
        for stock in top_25_negative:
            email_body += f"Stock: {stock[0]}, Price Change: {stock[1]['price_change']}\n"
            
        email_body += "\n\nNoted: The spreadsheet did not include the information about CXA market, and it's subject to 20 mins delay compared to the actual market."
        email_body += "\n\nBest Regards,"
        email_body += "\nWayne"
    return email_body


def send_csv_email():
    email_group = 'xxx@gmail.com'
    
    sever = smtplib.SMTP('smtp-mail.outlook.com', port=587)
    sever.starttls()

    sever.login('xxx@180markets.com.au', 'xxx')
    msg = MIMEMultipart()
    
    
    msg['From'] = 'xxx@180markets.com.au'
    msg['To'] = email_group  # Make sure email_group is defined
    msg['Subject'] = "Top Volume List"
    print(msg.as_string()) 


    email_body = generate_email_body()  # Make sure generate_email_body() is defined
    msg.attach(MIMEText(email_body, 'plain'))

    # List of CSV files to attach
    csv_files = ['sorted_volume_stocks.csv', 'sorted_stocks_positive.csv', 'sorted_stocks_negative.csv']

    # Attach each CSV file
    for csv_file in csv_files:
        with open(csv_file, 'r') as f:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(f.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename="{csv_file}"')
            msg.attach(part)

    # Send the message via the server
    sever.send_message(msg)
    print('email sent')
    # Terminate the SMTP session
    sever.quit()

send_csv_email()

Content-Type: multipart/mixed; boundary="===============3555316629543062382=="
MIME-Version: 1.0
From: wayne@180markets.com.au
To: waynewayne0707@gmail.com
Subject: Top Volume List

--===============3555316629543062382==

--===============3555316629543062382==--

email sent


# main.py

In [ ]:
def main():
    renew_industry_data()
    update_stock_data()
    process_data()
    send_csv_email()
    print("All tasks completed successfully.")

if __name__ == "__main__":
    main()